The purpose of this script is to scrape match information from Ballchasing.com. 

In [ ]:
import json
import requests
import pandas as pd

from datetime import date

In [ ]:
api_token = 'INSERT API TOKEN HERE'
api_url_base = 'https://ballchasing.com/api/replays'

headers = {'Authorization':api_token}

The logic in the below cell is abominable. To paraphrase a meme I saw once:

**When I wrote this code, only god and I knew how it worked. Now, after a few days, only he knows.**

Most of the ugliness comes from not using variables for each parsed datapoint. I should have declared those variables at the beginning of the loop,

e.g. blue_name = data['list'][match]['blue]['players'][0]['name'],

but alas, I did not. The rest of the unreadability comes from very basic error handling. 

I chose to designate player_1 as the uploader, who *must* be a steam user. Player 2 is then the player that does not have a name equal to player 1. This odd logic is due to the fact that the uploader could be either on the blue team *or* the orange team, so they must be compared to identify each player.

In [ ]:
url = api_url_base
name_key = 'name'
dataList = []


for page in range(1,200):
    
    response = requests.get(url, headers={'Authorization':api_token}, params = {'playlist':'ranked-duels','count':'200'})
    data = response.json()

    for match in range(len(data['list'])):
        
        if name_key in data['list'][match]['blue']['players'][0] and name_key in data['list'][match]['orange']['players'][0]:
            
            match_date = data['list'][match]['date']
            player1_name = data['list'][match]['uploader']['name']
            player1_steam_id = data['list'][match]['uploader']['steam_id']

            player2_name = data['list'][match]['blue']['players'][0]['name'] if data['list'][match]['blue']['players'][0]['name'] != player1_name else data['list'][match]['orange']['players'][0]['name']
            player2_platform = data['list'][match]['blue']['players'][0]['id']['platform'] if data['list'][match]['blue']['players'][0]['name'] != player1_name else data['list'][match]['orange']['players'][0]['id']['platform']
            
            if (data['list'][match]['blue']['players'][0]['name'] != player1_name and data['list'][match]['blue']['players'][0]['id']['platform'] == 'steam'):
            
                player2_steam_id = data['list'][match]['blue']['players'][0]['id']['id']
                
            elif (data['list'][match]['blue']['players'][0]['name'] == player1_name and data['list'][match]['orange']['players'][0]['id']['platform'] == 'steam'):
                
                player2_steam_id = data['list'][match]['orange']['players'][0]['id']['id']
                
            else:
                
                player2_steam_id = 'null'
            
            player1_outcome = '0' if 'Loss' in data['list'][match]['replay_title'] else '1' if 'Win' in data['list'][match]['replay_title'] else 'null'

            keys = ['match_date','player1_name', 'player1_steam_id','player2_name','player2_platform','player2_steam_id','player1_outcome']
            values = [match_date, player1_name, player1_steam_id, player2_name, player2_platform, player2_steam_id, player1_outcome]
            dataList.append(dict(zip(keys,values)))
            
        else:
            continue
        
    url = data['next']

In [ ]:
print(len(dataList))

In [ ]:
today = date.today()
date_today = today.strftime("%b-%d-%Y")

In [ ]:
df = pd.DataFrame(dataList)
df.to_excel('Ballchasing_data_' + date_today + '.xlsx')